In [40]:
%run PATTERN_OBSERVER.ipynb

In [41]:
from collections import namedtuple
from pathlib import Path
from functools import reduce
import pandas as pd

In [75]:
class DATA_CONTAINER(IObserver,ISubject):
    ####################################
    # static
    ####################################
    # standardized file names
    BT_path     = namedtuple('BT_path',['backtest_name','subdir','filename'])
    _dev        = BT_path('./','./',None)
    _daily      = BT_path(None,'./daily','summary_YYYYMMDD')
    _extraday   = BT_path(None,'./extraday','BOOK_YYYYMMDD')
    _intraday   = BT_path(None,'./intraday','BOOK_YYYYMMDD')
    _factor     = BT_path(None,'./factor','BOOK_YYYYMMDD')

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name='DATA_CONTAINER',
        filetype='dev',
        applied_backtest_path=r'C:\Users\ahkar\OneDrive\Documents\Data\B3',
        applied_date_from=None,
        applied_date_to=None,
        applied_return_type=None,
        selected_backtests={
            'AZUL4.SA': Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/AZUL4.SA.csv'),
            'EMBR3.SA': Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/EMBR3.SA.csv')
        },
        selected_books={
            'Trading': 'Trading',
            'Quote': 'Quote',
            'MM2': 'MM2',
            'Hedge': 'Hedge',
            'Hit': 'Hit'
        }
        ):
        self.name=name
        self.reference={
            'applied_backtest_path':applied_backtest_path,        # observes changes to `SETTINGS|backtest_path`
            'applied_date_from':applied_date_from,                # observes changes to `SETTINGS|date_from`
            'applied_date_to':applied_date_to,                    # observes changes to `SETTINGS|date_to`
            'applied_return_type':applied_return_type,            # observes changes to `SETTINGS|return_type`
            'selected_backtests':selected_backtests,              # observes changes to `BACKTEST_VIEWER|selected_backtests`
            'selected_books':selected_books,                      # observes changes to `BACKTEST_VIEWER|selected_books`
        }
            
        self.data_dico={} # container for all data
        self._observers=set() # for observer pattern
        self._logging() # show internals
        '''
        on backtest_widget apply button
            update DATA_CONTAINER.data dictionary
            update DATA_CONTAINER.selected_backtests
            
        on book_widget apply button
            update DATA_CONTAINER.selected_books
            
        post updating, update all visible plots
        '''
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self) -> None:
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,self.applied_settings)

    # observer
    def react(self,subject_name,subject_info):
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)

        # SETTINGS changed
        if subject_name=='SETTINGS':
            if self.reference['applied_backtest_path']!=subject_info['backtest_path']:
                self.data_dico.clear() # empty data_dico if filepath changed
            
            # update
            self.reference['applied_backtest_path']=subject_info['backtest_path']
            self.reference['applied_date_from']=subject_info['date_from']
            self.reference['applied_date_to']=subject_info['date_to']
            self.reference['applied_return_type']=subject_info['return_type']
        
        # BACKTEST_SELECTOR changed
        if subject_name=='BACKTEST_SELECTOR':
            self.reference['selected_backtests']=subject_info

            # update data_dico
            for (option_name,option_path),checked in self.reference['selected_backtests'].items():
                if checked:
                    self._add_key(option_name,option_path) # make sure present
                else:
                    self._remove_key(option_name,option_path) # make sure not present


        # BOOK_SELECTOR changed
        if subject_name=='BOOK_SELECTOR':
            self.reference['selected_books']=[k for k,v in subject_info.items() if v] # return checked values

        # logging
        self._logging()
        
    def _logging(self):
        print(self.name,':','REFERENCE',':','DATA_DICO SIZE =',len(self.data_dico))
        for k,v in self.reference.items():
            print(self.name,':','REFERENCE',':',k,':',v)

    ####################################
    # own stuff
    ####################################
    '''
    d=DATA_CONTAINER()
    print(d.get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),datatype='dev'))
    print(d.get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),datatype='daily'))
    print(d.get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),datatype='intraday'))
    print(d.get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),datatype='extraday'))
    print(d.get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),datatype='factor'))
    '''
    def get_filepath(self,option_name,dir_path,datatype='dev'):
        BT_path_tuple=eval('DATA_CONTAINER.'+datatype+'()') # get BT_path tuple
#        tup=[Path(self.reference['applied_backtest_path'])]+list(BT_path_tuple) # build tuple
        tup=[dir_path.parent]+list(BT_path_tuple) # build tuple
        tup=[option_name if x==None else x for x in tup] # overwrite None in BT_path_tuple with backtest_name
        filepath=reduce(lambda x,y:x/y,tup) # combine into single Path object
        filepath=Path(str(filepath) + '.csv') # append .csv suffix
        return filepath
        
    '''
    d=DATA_CONTAINER()
    d._add_key('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'))
    d._add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    d._add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    d._add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    '''
    def _add_key(self,option_name,option_path,filetype='dev'):
        # make sure data is populated
        if option_name in self.data_dico:
            print(self.name,':','KEYS =',len(self.data_dico),':',option_name,'PASS - ALREADY HERE')
        else:
            print(self.name,':','KEYS =',len(self.data_dico),':',option_name,'ADDED')
            self.data_dico[option_name]=pd.read_csv(self.get_filepath(option_name,option_path,filetype))
    
    def _remove_key(self,option_name,option_path):
        # make sure data removed from data_dico
        if option_name in self.data_dico:
            print(self.name,':','KEYS =',len(self.data_dico),':',option_name,'REMOVED')
            del self.data_dico[option_name] # remove from dico
        else:
            print(self.name,':','KEYS =',len(self.data_dico),':',option_name,'PASS - ALREADY NOT')

    @classmethod
    def dev(self):
        return DATA_CONTAINER._dev
    
    @classmethod
    def daily(self):
        '''
        purpose
             extraday summary --> macro extraday overview

        dump frequency
            once a day at end of day

        example files
            ./backtest_name/daily/summary_yyyymmdd.csv

        contents of each daily file
            book|pnl|stock volume|future volume|fees
            All|.|.|.|.
            Trading|.|.|.|.
            Quote|.|.|.|.
            Hedge|.|.|.|.

        used to compute summary for entire backtest
            book|return bps|sharpe|daily pnl|daily stock volume|daily future volume|fee bps
            All|.|.|.|.|.|.
            Trading|.|.|.|.|.|.
            Quote|.|.|.|.|.|.
            Hedge|.|.|.|.|.|.
        '''
        return DATA_CONTAINER._daily
    
    @classmethod
    def extraday(self):
        '''
        purpose
            extraday behavioural analysis --> customised macro extraday overview

        dump frequency
            once a day at end of day
            reference symbol `Symbol` is used to initialize the dump

        example files
            ./backtest_name/extraday/Trading_yyyymmdd.csv
            ./backtest_name/extraday/Quote_yyyymmdd.csv
            ./backtest_name/extraday/Hedge_yyyymmdd.csv

        contents of each daily file
            TimeStamp|Symbol|PnlTotal|PnlJour|PnlVeille|OpenNom|OpenBidNom|OpenAskNom|...
            EOD|VALE3.SA|.|.|.|.|.|.|...
        '''
        return DATA_CONTAINER._extraday
    
    @classmethod
    def intraday(self):
        '''
        purpose
            intraday behavioural analysis --> customised macro intraday overview

        dump frequency
            n-minutely snapshots throughout the day
            reference symbol `Symbol` is used to initialize the dump

        example files
            ./backtest_name/intraday/Trading_yyyymmdd.csv
            ./backtest_name/intraday/Quote_yyyymmdd.csv
            ./backtest_name/intraday/Hedge_yyyymmdd.csv

        contents of each daily file
            TimeStamp|Symbol|PnlTotal|PnlJour|PnlVeille|OpenNom|OpenBidNom|OpenAskNom|...
            10:00:00|VALE3.SA|.|.|.|.|.|.|...
            10:01:00|VALE3.SA|.|.|.|.|.|.|...
            ...
            16:59:00|VALE3.SA|.|.|.|.|.|.|...
            17:00:00|VALE3.SA|.|.|.|.|.|.|...
        '''
        return DATA_CONTAINER._intraday

    @classmethod
    def factor(self):
        '''
        purpose
            extraday factor analysis --> customised macro factor analysis

        dump frequency
            once a day at end of day
            dump on all symbols

        example files
            ./backtest_name/extraday/Trading_yyyymmdd.csv
            ./backtest_name/extraday/Quote_yyyymmdd.csv
            ./backtest_name/extraday/Hedge_yyyymmdd.csv

        contents of each daily file
            Symbol|PnlTotal|PnlJour|PnlVeille|ExecNom|MedLongNom|MaxLongLongNom|MedOpenBidNom|MedOpenAskNom|...
            sym_0|
            sym_1|
            ...
            sym_n|
        '''
        return DATA_CONTAINER._factor
    
        '''
    
            self.raw_data=
        self.filtered_data=self.apply_date_filters()
        

    def apply_date_filters(self):
        # check date validity
        
        # remember filter dates applied
        min_date=self.settings['date_from'].value
        max_date=self.settings['date_to'].value

        # apply date filter
        if min_date==None and max_date==None:
            print('no filter')
            self.filtered_data=self.raw_data
        elif min_date!=None and max_date==None:
            print('filter min_date')
            self.filtered_data=self.raw_data[self.raw_data.Date>=min_date]
        elif min_date==None and max_date!=None:
            print('filter max_date')
            self.filtered_data=self.raw_data[self.raw_data.Date<=max_date]
        else:
            print('filter both')
            self.filtered_data=self.raw_data[self.raw_data.Date.between(min_date,max_date)]
            
        # debugging
        print(min_date)
        print(max_date)
        print(self.raw_data.shape)
        print(self.filtered_data.shape)
    '''


In [68]:
'''
d=DATA_CONTAINER()
'''
None

In [6]:
# help(DATA_CONTAINER.daily)

In [7]:
# help(DATA_CONTAINER.extraday)

In [8]:
# help(DATA_CONTAINER.intraday)

In [9]:
# help(DATA_CONTAINER.factor)